## Mishandra: Python

This notebook demonstrates how to save data to a Cassandra cluster and retrieve it back.

In [1]:
%load_ext autoreload
%autoreload 2
 
from IPython.display import display, Video
    
import sys, os, random
import numpy as np
from PIL import Image
    
import mishandra
mishandra.__version__

'0.1.0'

#### Create a Mishandra session and print Cassandra keyspaces
```MishandraSession``` class handles communication with Cassandra cluster and contains routines for data I/O.

A ```keyspace``` in Cassandra is a top-level database object that controls replication.
Each keyspace can contain multiple tables.

A Mishandra ```Collection``` consists of multiple ```Packs```.

A ```Collection``` corresponds to a table and a ```Pack``` corresponds to a table row.

Each ```Pack``` consists of multiple ```FrameSets```. A bunch of ```FrameSets``` define the "width" of a ```Collection```. A single ```FrameSet``` corresponds to a cell of a Cassandra table.

A ```FrameSet``` is literally nothing more than a set of ```Frames```. A single ```Frame``` is a valid ```FrameSet```.

A ```Frame``` is a data snapshot at some point in time.

```FrameSets``` are stored as binary blobs. Normally, ```FrameSets``` are serialized and deserialized independently, but if a ```FrameSet``` is too big to be stored in a single cell, a ```Pack``` of such ```FrameSets``` can be split into even binary blobs and saved to corresponding cells.

Serialized binary ```FrameSets``` can be stacked together and then unpacked into a list of ```FrameSets```.

The stored data, for example, could be a 3d mesh sequence with shared or not shared topology, a point cloud, a video splitted into images and audio chunks, or any other data sequence that fits Mishandra's data representation format. 

In [2]:
session = mishandra.MishandraSession(contact_points=["127.0.0.1"], verbose=True)

Cannot connect to cluster
('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})
Mishandra session created


A session object provides a set of listed below functions for basic keyspace and collection manipulations.
#### Set an active keyspace. Create a keyspace if not exists
An active keyspace cat be specified once and used for further collection-related operations.

In [3]:
import mishandra

session.set_keyspace('dogs', use_simple_strategy=True, replication_factor=2)

AttributeError: 'NoneType' object has no attribute 'execute'

#### Delete a keyspace and its data if exists

In [247]:
session.delete_keyspace('dogs')

Keyspace misha dropped
Keyspaces:
dogs (durable_writes: True, replication: {'class': 'org.apache.cassandra.locator.SimpleStrategy', 'replication_factor': '2'})
   table hou_misha (rows: 142)
   table misha (rows: 16)


#### Create a collection if not exists
Collections are data agnostic. It means that a collection is not tied to data type and we can place there anything that was packed through a Mishandra interface.

In [13]:
session.create_collection('misha')

#### Delete a collection if exists
A keyspace can be also passed explicitly as an argument.

In [11]:
session.delete_collection('misha', keyspace='dogs')

Table dogs.misha dropped
Keyspaces:
dogs (durable_writes: True, replication: {'class': 'org.apache.cassandra.locator.SimpleStrategy', 'replication_factor': '2'})


#### Clear a collection if exists

In [86]:
session.clear_collection('misha')

Table dogs.misha truncated
Keyspaces:
dogs (durable_writes: True, replication: {'class': 'org.apache.cassandra.locator.SimpleStrategy', 'replication_factor': '2'})
   table misha (rows: 0)


#### Load a sequence of meshes from a directory of ```.obj``` files and visualize

In [4]:
# Make a list of meshes. Take all '.obj' files from a directory by default.
meshes = mishandra.trimesh.from_directory(
    os.path.join('..', 'test_data', 'pighead'),
    verbose=True
)

# Create a visualizer
renderer = mishandra.utils.OffscreenRenderer()

# Render meshes to a list of images
images = [renderer.update_scene(mesh).render()[0] for mesh in meshes]

# Make video from images and display
display(Video(mishandra.utils.make_video(images), embed=True))

16 meshes loaded (0001.obj..0016.obj)


#### Convert a sequence of meshes to a Pack

Mishandra converts data to intermediate protobuf representation. All the heavy proto fields are binary blobs, which gives fast access and compact storage.
Here we pack 16 meshes to a single frame which contains 16 objects.

In [16]:
# Make a pack from meshes
# [[[mesh, mesh]]] 1 frameset of 1 frame of 2 objects")
# [[[mesh], [mesh]]] 1 frameset of 2 frames of 1 object")
# [[[mesh]], [[mesh]]] 2 framesets of 1 frame of 1 object")
pack = mishandra.trimesh.pack([[meshes]], id=0)

# Inspect the structure
mishandra.proto.print_fields(pack, repeated_fields_limit=2)

FrameSet
|  id 0
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (1)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (16)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  nPV 3
|  |  |  |  |  |  

You can notice that internally the ```Pack``` itself is a ```FrameSet```, but it shouldn't confuse you.

It makes more sense to represent the animation sequence as a ```FrameSet``` since a ```FrameSet``` implies variation in time:

In [17]:
pack = mishandra.trimesh.pack([[[mesh] for mesh in meshes]], id=0)

mishandra.proto.print_fields(pack, repeated_fields_limit=2)

FrameSet
|  id 0
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (16)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  nPV 3
|  |  |  |  |  |  

We can see that the total size of the ``FrameSet`` is approximately 1MB which is acceptable for a Cassandra cell.
But if it was an order of magnitude larger, it would have been better to store the sequence in multiple packs:

In [18]:
packs = [
    mishandra.trimesh.pack(
        [[[mesh]]],
        id = i
    ) for i, mesh in enumerate(meshes)
]

print(f"Number of Packs: {len(packs)}")

pack = random.choice(packs)
print(f"Pack {pack.id}:")
mishandra.proto.print_fields(pack, repeated_fields_limit=2)

Number of Packs: 16
Pack 8:
FrameSet
|  id 8
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (1)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  

Let's suppose we have two sequences of meshes:

In [19]:
meshes_1, meshes_2 = meshes, meshes

We could pack them as follows:

In [20]:
# Convert two mesh sequences to a list of Packs each of which contains 2 FrameSets of length 1
packs = [
    mishandra.trimesh.pack(
        [[[mesh_1]], [[mesh_2]]],
        id=i,
    ) for i, (mesh_1, mesh_2) in enumerate(zip(meshes_1, meshes_2))
]

print(f"Number of Packs: {len(packs)}")

pack = random.choice(packs)
print(f"Pack {pack.id}:")
mishandra.proto.print_fields(pack, repeated_fields_limit=2)

Number of Packs: 16
Pack 13:
FrameSet
|  id 13
|  frames (0)
|  frameSets (2)
|  |  0:id 0
|  |  0:frames (1)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |

There may be fields that are present in all packs but don't vary. We could mark them as cached and store in a single ```Master Pack``` only:

In [28]:
master_pack = mishandra.trimesh.pack(
    [[[meshes[0]]]],
    id = 0,
    is_master = True
)

# Mark all primitiveSet fields in all Geometry objects as cached
mishandra.proto.mark_cached(master_pack, "Object.images", verbose=True)
# This function works recursively by default. We could also pick a specific child field down the hierarchy and go from there
mishandra.proto.mark_cached(master_pack.frameSets[0].frames[0], "Object.images")

packs = [
    mishandra.trimesh.pack(
        [[[mesh]]],
        id = i+1,
        master_pack = master_pack, # leave marked fields empty
    ) for i, mesh in enumerate(meshes[1:])
]
packs = [master_pack] + packs

print(f"Number of Packs: {len(packs)}")

print(f"Master Pack:")
mishandra.proto.print_fields(master_pack, repeated_fields_limit=2)

pack = random.choice(packs)
print(f"Random Pack:")
mishandra.proto.print_fields(pack, repeated_fields_limit=2)

Object.images: 1 fields marked as cached
Number of Packs: 16
Master Pack:
FrameSet
|  id 0
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (1)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|

#### Save Packs to a Cassandra cluster

In [29]:
# We don't want to accumulate tombstones, so we truncate the Cassandra table. Deleting a table is ok also
session.clear_collection('misha', keyspace='dogs')

for pack in packs:
    session.save_pack_to_cluster(
        keyspace='dogs',
        collection='misha',
        id=pack.id,
        pack=pack,
        verbose=True
    )

session.print_keyspaces()

Table dogs.misha truncated
Keyspaces:
dogs (durable_writes: True, replication: {'class': 'org.apache.cassandra.locator.SimpleStrategy', 'replication_factor': '2'})
   table hou_misha (rows: 39)
   table misha (rows: 0)
Id: 0, Storage Mode: 1, Size: 440KB, Blobs: 1
Id: 1, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 2, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 3, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 4, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 5, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 6, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 7, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 8, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 9, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 10, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 11, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 12, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 13, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 14, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 15, Storage Mode: 1, Size: 439KB, Blobs: 1
Keyspaces:
dogs (durable_writes: True, 

If a single ```FrameSet``` is too large to be stored in a single cell, it's more appropriate to split a ```Pack``` into small even parts by setting the ```as_even_blobs``` argument to ```True```.

We can have ```Packs``` up to 512MB in size this way, but it also means we have to deserialize an entire ```Pack``` in order to access data from a single ```FrameSet```. It's not a problem if all ```FrameSets``` of a ```Pack``` are needed to be in memory anyway.

In [31]:
# We don't want to accumulate tombstones, so we truncate the Cassandra table. Deleting a table is ok also
session.clear_collection('misha', keyspace='dogs')

for pack in packs:
    session.save_pack_to_cluster(
        keyspace='dogs',
        collection='misha',
        id=pack.id,
        pack=pack,
        as_even_blobs=True, # if the size of a pack exceeds 1MB, split the pack into pieces 1MB each
        verbose=True
    )

session.print_keyspaces()

Table dogs.misha truncated
Keyspaces:
dogs (durable_writes: True, replication: {'class': 'org.apache.cassandra.locator.SimpleStrategy', 'replication_factor': '2'})
   table hou_misha (rows: 39)
   table misha (rows: 0)
Id: 0, Storage Mode: 1, Size: 440KB, Blobs: 1
Id: 1, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 2, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 3, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 4, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 5, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 6, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 7, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 8, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 9, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 10, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 11, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 12, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 13, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 14, Storage Mode: 1, Size: 439KB, Blobs: 1
Id: 15, Storage Mode: 1, Size: 439KB, Blobs: 1
Keyspaces:
dogs (durable_writes: True, 

#### Load Packs from a Cassandra cluster
```MishandraSession.load_pack_from_cluster``` function loads a single ```Pack``` into memory.

In [32]:
# Load the master pack from a collection
master_pack = session.load_pack_from_cluster(collection='misha', keyspace='dogs', id=0)

mishandra.proto.print_fields(master_pack, repeated_fields_limit=2)

FrameSet
|  id 0
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (1)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  nPV 3
|  |  |  |  |  |  n

```MishandraSession.load_pack_range_from_cluster``` loads a range of packs:

In [33]:
# It's ok to request a range with a margin
packs = session.load_pack_range_from_cluster(collection='misha', keyspace='dogs', id_from=0, id_to=20, verbose=True)

mishandra.proto.print_fields(packs[-1], repeated_fields_limit=2)

pack 0 loaded from dogs misha
pack 1 loaded from dogs misha
pack 2 loaded from dogs misha
pack 3 loaded from dogs misha
pack 4 loaded from dogs misha
pack 5 loaded from dogs misha
pack 6 loaded from dogs misha
pack 7 loaded from dogs misha
pack 8 loaded from dogs misha
pack 9 loaded from dogs misha
pack 10 loaded from dogs misha
pack 11 loaded from dogs misha
pack 12 loaded from dogs misha
pack 13 loaded from dogs misha
pack 14 loaded from dogs misha
pack 15 loaded from dogs misha
16 rows loaded
FrameSet
|  id 15
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (1)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:

 #### Make a sequence of meshes from packs and visualize

In [34]:
# Make meshes from packs
loaded_meshes = [mishandra.trimesh.unpack(pack, master_pack=master_pack, flatten=True) for pack in packs]

# Make video from images in reversed order and display
images = [renderer.update_scene(mesh).render()[0] for mesh in loaded_meshes][::-1]
display(Video(mishandra.utils.make_video(images), embed=True))

#### Save a Pack to file
It's also easy to save a pack to a binary file:

In [35]:
pack = mishandra.trimesh.pack([[[mesh] for mesh in meshes]], id=0)

# Save the last pack to file
session.save_pack_to_file('dog.misha', pack)

and to load a Pack from file:

In [36]:
pack = session.load_pack_from_file('dog.misha')

loaded_meshes = mishandra.trimesh.unpack(pack, flatten=True)

mishandra.proto.print_fields(pack, repeated_fields_limit=2)

images = [renderer.update_scene(mesh).render()[0] for mesh in loaded_meshes][::-1]
display(Video(mishandra.utils.make_video(images), embed=True))

FrameSet
|  id 0
|  frames (0)
|  frameSets (1)
|  |  0:id 0
|  |  0:frames (16)
|  |  |  0:id 0
|  |  |  0:name ''
|  |  |  0:timestamp 0.0
|  |  |  0:objects (1)
|  |  |  |  0:id 0
|  |  |  |  0:name ''
|  |  |  |  0:text ''
|  |  |  |  0:camera
|  |  |  |  |  type PERSPECTIVE
|  |  |  |  |  extrinsic
|  |  |  |  |  |  id 0
|  |  |  |  |  |  transform (0)
|  |  |  |  0:transform
|  |  |  |  |  id 0
|  |  |  |  |  transform (0)
|  |  |  |  0:geometry (1)
|  |  |  |  |  0:id 0
|  |  |  |  |  0:name ''
|  |  |  |  |  0:pointSet
|  |  |  |  |  |  P (8658) 33.82KB float32 (-1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  cachedFields []
|  |  |  |  |  |  lowPrecisionFields []
|  |  |  |  |  0:primitiveSet
|  |  |  |  |  |  type POLYGON
|  |  |  |  |  |  faces (17304) 67.59KB uint32 (-1, -1)
|  |  |  |  |  |  uv (34608) 135.19KB float32 (-1, -1, -1, 2)
|  |  |  |  |  |  N (51912) 202.78KB float32 (-1, -1, 3)
|  |  |  |  |  |  groupNames []
|  |  |  |  |  |  nPV 3
|  |  |  |  |  |  

Mishandra saves and loads data in little endian order. No worries about hardware compatibility.